## NOTE
The Purpose of my repos is to learn, in this repo I try to learn how to use TensorFlow Keras, Hyperopt, MLflow to develop a deep learning model

It includes the following steps: 
- STEP 1: DATA PREP
Load and preprocess data 
- STEP 2: Neural Network Model
        Part 1. Create a neural network model with TensorFlow Keras and view training with inline TensorBoard
        Part 2. Perform automated hyperparameter tuning with Hyperopt and MLflow and use autologging to save results
        Part 3. Use the best set of hyperparameters to build a final model
        Part 4. Register the model in MLflow and use the model to make predictions

This repo follow instructions notebooks provided on DataBricks websites

In [2]:
#tensorflow libraries, mlflow
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import mlflow
import mlflow.keras
import mlflow.tensorflow

## DATA PREP
Using California Housing dataset scikit-learn.

### Load and train-test-split dataset

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

df_cal_housing = fetch_california_housing()

#split train-test
X_train, X_test, y_train, y_test = train_test_split(df_cal_housing.data, df_cal_housing.target, test_size=0.2)

In [10]:
X_train

array([[   2.1447    ,   26.        ,    3.48595041, ...,    3.50413223,
          33.79      , -118.27      ],
       [   4.1739    ,   36.        ,    5.45962733, ...,    2.86749482,
          34.09      , -117.71      ],
       [   3.1641    ,   17.        ,    5.58348624, ...,    2.6733945 ,
          35.35      , -120.49      ],
       ...,
       [   2.63      ,   52.        ,    4.51311953, ...,    3.29737609,
          34.07      , -117.75      ],
       [   3.3326    ,   52.        ,    3.89162562, ...,    3.60098522,
          34.12      , -118.2       ],
       [   8.0784    ,   52.        ,    6.88219178, ...,    2.67945205,
          33.77      , -117.87      ]])

### Scale features
Feature scaling is important when working with neural networks, we will use StandardScaler

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) #scaler is fitted by X_train already, using transform only

## Neural Network Model

#### Part 1. Create model and view TensorBoard in notebook

In [12]:
#define model
def create_model():
    model = Sequential()
    #relu = rectified linear activation function - looks and acts like a linear function, 
    #but is, in fact, a nonlinear function allowing complex relationships in the data to be learned.
    #Dense is layer, is deeply connected with its preceding layer 
    # which means the neurons of the layer are connected to every neuron of its preceding layer. 
    model.add(Dense(20, input_dim=8, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='linear'))
    return model

In [15]:
#compile the model
model = create_model()
#----
model.compile(loss='mse',
             optimizer='Adam',
             metrics=['mse'])

#### callbacks
callbacks are the special utilities or functions that are executed during training at given stages of the training procedure. Callbacks can help you prevent overfitting, visualize training progress, debug your code, save checkpoints, generate logs, create a TensorBoard, etc. There are many callbacks readily available in TensorFlow, and you can use multiple. 
https://blog.paperspace.com/tensorflow-callbacks/

In [17]:
#create callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#directory
experiment_log_dir = './DB/tb'
checkpoint_path = './DB/keras_checkpoint_weights.ckpt'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=experiment_log_dir)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='loss', mode='min', patience=3)

history = model.fit(X_train, y_train, validation_split=.2, epochs=50, callbacks=[tensorboard_callback,
                                                                                 model_checkpoint,
                                                                                early_stopping])

Epoch 1/50
409/413 [============================>.] - ETA: 0s - loss: 0.2958 - mse: 0.2958
Epoch 1: val_loss improved from inf to 0.29848, saving model to ./DB/keras_checkpoint_weights.ckpt
INFO:tensorflow:Assets written to: ./DB/keras_checkpoint_weights.ckpt/assets
413/413 [==============================] - 3s 8ms/step - loss: 0.2964 - mse: 0.2964 - val_loss: 0.2985 - val_mse: 0.2985
Epoch 2/50
412/413 [============================>.] - ETA: 0s - loss: 0.2966 - mse: 0.2966
Epoch 2: val_loss improved from 0.29848 to 0.29469, saving model to ./DB/keras_checkpoint_weights.ckpt
INFO:tensorflow:Assets written to: ./DB/keras_checkpoint_weights.ckpt/assets
413/413 [==============================] - 3s 8ms/step - loss: 0.2964 - mse: 0.2964 - val_loss: 0.2947 - val_mse: 0.2947
Epoch 3/50
413/413 [==============================] - ETA: 0s - loss: 0.2973 - mse: 0.2973
Epoch 3: val_loss improved from 0.29469 to 0.29328, saving model to ./DB/keras_checkpoint_weights.ckpt
INFO:tensorflow:Assets wri

413/413 [==============================] - 3s 8ms/step - loss: 0.2777 - mse: 0.2777 - val_loss: 0.2906 - val_mse: 0.2906
Epoch 28/50
407/413 [============================>.] - ETA: 0s - loss: 0.2796 - mse: 0.2796
Epoch 28: val_loss did not improve from 0.28212
413/413 [==============================] - 3s 8ms/step - loss: 0.2786 - mse: 0.2786 - val_loss: 0.2973 - val_mse: 0.2973
Epoch 29/50
409/413 [============================>.] - ETA: 0s - loss: 0.2773 - mse: 0.2773
Epoch 29: val_loss did not improve from 0.28212
413/413 [==============================] - 3s 7ms/step - loss: 0.2777 - mse: 0.2777 - val_loss: 0.2873 - val_mse: 0.2873
Epoch 30/50
409/413 [============================>.] - ETA: 0s - loss: 0.2775 - mse: 0.2775
Epoch 30: val_loss did not improve from 0.28212
413/413 [==============================] - 3s 7ms/step - loss: 0.2790 - mse: 0.2790 - val_loss: 0.2857 - val_mse: 0.2857
Epoch 31/50
408/413 [============================>.] - ETA: 0s - loss: 0.2770 - mse: 0.2770
Epoc

#### TensorBoard commands

In [21]:
%load_ext tensorboard
%tensorboard --logdir $experiment_log_dir

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
